In [2]:
# Methods for display of arbitrary HTML
from IPython.display import HTML

# Methods for capture of secure info (like registration keys)
import getpass

# Methods for capture of data returned by GET request
import requests

# Methods for turning API result into pandas DataFrame
import json
import numpy as np
import pandas as pd

# Methods for plotting
import bokeh.plotting as bp

# Display result in an iframe
def show_iframe(url, height=400, width=1000):
    display_string = '<iframe src={url} width={w} height={h}></iframe>'.format(url=url, w=width, h=height)
    return HTML(display_string)

show_iframe('https://api.bls.gov/publicAPI/v1/timeseries/data/CFU0000008000', height=100)

In [3]:
show_iframe('https://data.bls.gov/registrationEngine/')

In [4]:
# Capture key
reg_key = getpass.getpass('Enter Registration Key: ')

In [5]:
show_iframe('https://www.bls.gov/help/hlpforma.htm#CX', height=400)

In [19]:
# Build series; unadjusted furniture purchases, highest level of education by any household member, completed bachelor's degree, process code
series = 'CXUFURNITURLB1408M'

def capture_request(series, start, end, key=reg_key):
    # Define base URL (for v2)
    url = 'https://api.bls.gov/publicAPI/v2/timeseries/data/'
    # Concatenate registration key
    url += '?registrationkey={key}'.format(key=key)
    # Cast data request as JSON
    data = json.dumps({
        "seriesid": [series],
        "startyear": str(start),
        "endyear": str(end)
    })
    # Capture content type of request
    headers = {
        "Content-type": "application/json"
    }
    # Request data
    result = requests.post(url, data=data, headers=headers)
    return json.loads(result.text)

# Capture and display JSON representation of the data
json_data = capture_request(series, 2012, 2016)

json_data

{'Results': {'series': [{'data': [{'footnotes': [{}],
      'period': 'A01',
      'periodName': 'Annual',
      'value': '546',
      'year': '2016'},
     {'footnotes': [{}],
      'period': 'A01',
      'periodName': 'Annual',
      'value': '636',
      'year': '2015'},
     {'footnotes': [{}],
      'period': 'A01',
      'periodName': 'Annual',
      'value': '496',
      'year': '2014'},
     {'footnotes': [{}],
      'period': 'A01',
      'periodName': 'Annual',
      'value': '528',
      'year': '2013'},
     {'footnotes': [{}],
      'period': 'A01',
      'periodName': 'Annual',
      'value': '506',
      'year': '2012'}],
    'seriesID': 'CXUFURNITURLB1408M'}]},
 'message': [],
 'responseTime': 50,
 'status': 'REQUEST_SUCCEEDED'}

In [20]:
# Convert to DF
df_data = pd.DataFrame(json_data['Results']['series'][0]['data'])
print(df_data)

# Retain only the values and years (convert both values to int)
df_sub = df_data[['year', 'value']].astype(int)

# Set year to index
df_sub.set_index('year', inplace=True)

# Sort index
df_sub.sort_index(inplace=True)

df_sub

  footnotes period periodName value  year
0      [{}]    A01     Annual   546  2016
1      [{}]    A01     Annual   636  2015
2      [{}]    A01     Annual   496  2014
3      [{}]    A01     Annual   528  2013
4      [{}]    A01     Annual   506  2012


,value
year,
2012,506
2013,528
2014,496
2015,636
2016,546


In [26]:
# Create file path to hold figure we are about to make
bp.output_file('bls_api_fig.html')

# Create plotting figure
fig = bp.figure(plot_width=600, plot_height=300)

# Add a line to the figure
fig.line(df_sub.index, df_sub['value'], line_width=3, color='blue')

# Annotate plot
fig.title.text = "Furniture Spending Spiked in 2015 for Households with a Bachelor's-Degree Holder"
fig.xaxis.axis_label = "Year"
fig.yaxis.axis_label = "Furniture Expenditures"

# Show the figure
bp.save(fig)

show_iframe('bls_api_fig.html', width=650, height=350)